In [1]:
from presidio_analyzer import AnalyzerEngine
from presidio_anonymizer import AnonymizerEngine
from presidio_anonymizer.entities import OperatorConfig
from pyspark.sql.types import StringType
from pyspark.sql.functions import input_file_name, regexp_replace
from pyspark.sql.functions import col, pandas_udf
import pandas as pd
import os

StatementMeta(presido, 8, 1, Finished, Available)

In [2]:
#read sample file to get initial values
df = spark.read.option("header",True).csv("/MOCK_DATE.csv")
display(df)

StatementMeta(presido, 8, 2, Finished, Available)

SynapseWidget(Synapse.DataFrame, 7fa1da47-57de-485a-8611-aa10cfe25671)

In [3]:
from presidio_analyzer import AnalyzerEngine, PatternRecognizer, EntityRecognizer, Pattern, RecognizerResult

#set up analyzer and anonymizer
analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

 
#Based on simple example:https://microsoft.github.io/presidio/samples/python/presidio_notebook/
#Add names I want to anonymize.
titles_list = ["Roda", "Stirling","Terrijo", "Chrisse", "Leyla"]
titles_recognizer = PatternRecognizer(supported_entity="TITLE", deny_list=titles_list)

ip_pattern = Pattern(name="ip_pattern",regex="\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}", score = 0.5)
# Define the recognizer with one or more patterns
ip_recognizer = PatternRecognizer(supported_entity="IPADDRESS", patterns = [ip_pattern])


#Register Patterns
analyzer.registry.add_recognizer(titles_recognizer)
analyzer.registry.add_recognizer(ip_recognizer)


broadcasted_analyzer = sc.broadcast(analyzer)
broadcasted_anonymizer = sc.broadcast(anonymizer)

StatementMeta(presido, 8, 3, Finished, Available)

configuration file /home/trusted-service-user/cluster-env/env/lib/python3.8/site-packages/conf/default.yaml not found.  Using default config: {'nlp_engine_name': 'spacy', 'models': [{'lang_code': 'en', 'model_name': 'en_core_web_lg'}]}.

In [4]:
#can be expanded to use any of the following techniques:https://microsoft.github.io/presidio/anonymizer/#built-in-operators
from pyspark.sql.types import StringType
from pyspark.sql.functions import udf
@udf(returnType=StringType()) 
def at(text: str):
    analyzer = broadcasted_analyzer.value
    anonymizer = broadcasted_anonymizer.value
    #Add entities to analyze
    analyzer_results = analyzer.analyze(text=text, entities=["TITLE","EMAIL_ADDRESS", "IPADDRESS"] ,language="en")
    #Anonymize if one of the names match
    anonymized_results = anonymizer.anonymize(
        text=text,
        analyzer_results=analyzer_results,
        operators={"DEFAULT": OperatorConfig("replace", {"new_value": "<ANONYMIZED>"}),
        #EMAIL_ADDRESS is a built-in supported entity
        "EMAIL_ADDRESS": OperatorConfig("hash"),
        #Instead of built-in IP_ADDRESS entity, we are using a custom entity
        "IPADDRESS": OperatorConfig("mask", {"type": "mask", "masking_char" : "*", "chars_to_mask" : 12, "from_end" : True})
        },
    )
    return anonymized_results.text

StatementMeta(presido, 8, 4, Finished, Available)

In [5]:
#Register UDF
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType,StringType
anon = udf(at, StringType())


StatementMeta(presido, 8, 5, Finished, Available)

In [6]:
# apply the udf
anonymized_df = (df
    .withColumn("first_name", anon.func(col("first_name")))
    .withColumn("email", anon.func(col("email")))
    .withColumn("ip_address", anon.func(col("ip_address")))
    )

#Display Anonymized dataframe
display(anonymized_df)

StatementMeta(presido, 8, 6, Finished, Available)

SynapseWidget(Synapse.DataFrame, 13183f40-a46f-47ed-9225-03ff322523df)